Chow test for Cluster 0

In [49]:
import pandas as pd

In [50]:
cluster0 = pd.read_csv("data_stationnaires\Cluster0_Phase_II.csv")
cluster1 = pd.read_csv("data_stationnaires\Cluster1_Phase_II.csv")

In [51]:
cluster0.drop(columns=['Unnamed: 0'],inplace=True)
cluster1.drop(columns=['Unnamed: 0'],inplace=True)

cluster0.dropna(inplace=True)
cluster1.dropna(inplace=True)

In [52]:
cluster0['Date'] = pd.to_datetime(cluster0['Date'])

# Split the data
breakpoint = pd.Timestamp('2011-12-01')
df_pre = pd.DataFrame(cluster0[cluster0['Date'] < breakpoint])
df_post = pd.DataFrame(cluster0[cluster0['Date'] >= breakpoint])

In [53]:
import statsmodels.api as sm
import numpy as np

#model
def model(y,X):
    T = len(y)
    maxlags = int(4 * (T / 100)**(2/9)) 
    model = sm.OLS(y, X).fit()
    robust_se_model = model.get_robustcov_results(cov_type='HAC', maxlags=maxlags)

    while True:
        current_aic = robust_se_model.aic
        current_bic = robust_se_model.bic
        current_adj_r_squared = robust_se_model.rsquared_adj
        if max(robust_se_model.pvalues) > 0.05:
            # Remove the least significant variable
            worst_feature = int(np.argmax(robust_se_model.pvalues))
            worst_feature = X.columns[worst_feature]
            X_temp = X.drop(columns=[worst_feature])
            model_temp = sm.OLS(y, X_temp).fit()
            model_temp = model_temp.get_robustcov_results(cov_type='HAC', maxlags=maxlags)
            
            # Check if removing worsens AIC, BIC, or adjusted R^2
            if model_temp.aic < current_aic and model_temp.bic < current_bic and model_temp.rsquared_adj > current_adj_r_squared:
                X = X_temp
                robust_se_model = model_temp
            else:
                break  
        else:
            break
    return robust_se_model

In [54]:
import statsmodels.api as sm


X_pre = df_pre.drop(columns=['Date','Price'])
X_pre = sm.add_constant(X_pre)  # Using index as a regressor for simplicity
Y_pre = df_pre['Price']
model_pre = model(Y_pre, X_pre)

X_post = df_post.drop(columns=['Date','Price'])
X_post = sm.add_constant(X_post)  # Reset index for post model
Y_post = df_post['Price']
model_post = model(Y_post, X_post)

c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: divide

In [55]:
def chow_test(model_pre, model_post, df_pre, df_post, total_df):
    RSS0 = sum((total_df['Price'] - total_df['Price'].mean())**2)
    RSS1 = sum((df_pre['Price'] - model_pre.predict(sm.add_constant(df_pre.index)))**2)
    RSS2 = sum((df_post['Price'] - model_post.predict(sm.add_constant(df_post.index - df_post.index[0])))**2)
    n = len(total_df)
    k = model_pre.params.size  # Number of parameters estimated, typically includes the constant
    F = ((RSS0 - RSS1 - RSS2) / k) / ((RSS1 + RSS2) / (n - 2 * k))
    return F

# Total data
X_total = sm.add_constant(cluster0.index)
Y_total = cluster0['Price']
model_total = sm.OLS(Y_total, X_total).fit()

# Perform Chow Test
F_statistic = chow_test(model_pre, model_post, df_pre, df_post, cluster0)
print(f"Chow Test F-statistic: {F_statistic}")


ValueError: shapes (43,2) and (48,) not aligned: 2 (dim 1) != 48 (dim 0)

In [ ]:
from scipy.stats import f

n = len(cluster0)  # total number of observations
k = model_pre.params.size  # number of parameters estimated in one model

# Degrees of freedom
dfn = k  # number of parameters estimated
dfd = n - 2 * k  # total observations minus twice the number of estimated parameters

# Calculating the p-value from the F-distribution
p_value = 1 - f.cdf(F_statistic, dfn, dfd)
print(f"Chow Test p-value: {p_value}")


Chow Test p-value: 0.9150000136408195
